This script will get the omission, comission and intersection numbers from GABAM and MTBS.

In [2]:
import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
import geemap
import os
from google.cloud import storage
from google.cloud import client

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)

ee.Initialize()

os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)

In [3]:
#year of interest
years = [2004, 2005, 2009, 2010, 2013, 2015]

for year in years:
    
    #buffered fire polygons
    buffer = ee.FeatureCollection("users/spotter/fire_cnn/raw/2001_2023_buff").filter(ee.Filter.eq('Year',  year))   
    
    #grid used for predicting on and we will loop through to save memory
    grd = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_grid_20000")
    
    #for water masking
    water = ee.ImageCollection("JRC/GSW1_4/YearlyHistory")
    water = water.filterDate(str(year) + '-01-01', str(year) + '-12-31').max().select('waterClass').clip(grd.union())
    water = water.updateMask(water.eq(3))

    #modis land cover for now, we will switch this eventually
    lc = ee.ImageCollection("MODIS/061/MCD12Q1")

    #read in gabam which is a 30m random forest model
    gabam = ee.ImageCollection("projects/sat-io/open-datasets/GABAM")

    gabam = gabam.filterDate(str(year) + '-01-01', str(year) + '-12-31').mean()

    #clip lc
    lc = lc.filterDate(str(year) + '-01-01', str(year) + '-12-31').max().clip(grd.union())
    lc = lc.select('LC_Type1')

    #union to a single feature for the grd
    aoi = grd.union()

    
    #get modis active fire and merge with viirs
    Terra_Fire = ee.ImageCollection("MODIS/061/MOD14A1").filterBounds(aoi)
    Aqua_Fire = ee.ImageCollection("MODIS/061/MYD14A1").filterBounds(aoi)

    #merge and filter modis
    mod_fire = Terra_Fire.merge(Aqua_Fire).filterBounds(aoi).filterDate(str(year) + '-01-01', str(year) + '-12-31')
    
    #add day of year as band
    def add_bands(image):
        image = image.addBands(ee.Image(ee.Date(image.get('system:time_start')).getRelative('day','year').add(1)).clamp(1,366)).updateMask(image.select('FireMask').gte(7))
        return image.updateMask(image.select('constant').gt(60))


    mod_fire =  mod_fire.map(add_bands)

    mod_fire = mod_fire.select('constant').max().clip(aoi)


    #now buffer the buffered fire polygons, turn it to an image and merge with modis and viirs
    def to_year(f):
        return f.set({'Year': year})


    buffer = buffer.map(to_year)

    #turn vector to image
    buff_img = buffer.reduceToImage(
      properties= ['Year'],
      reducer= ee.Reducer.max()).clip(aoi).select(['max'], ['constant'])

    #merge modis and mtbs
    mod_mtbs = ee.ImageCollection([buff_img, mod_fire]).max()

    #only merge with viirs if the year is >= 2012, otherwise it does not exist
    if year >= 2012:

        #read in viirs
        viirs = ee.Image("users/spotter/fire_cnn/VIIRS/" + str(year))
        viirs = viirs.clip(aoi).select(['b1'], ['constant'])

        #merge all
        final = ee.ImageCollection([viirs, mod_mtbs]).max().updateMask(lc.neq(12).And(lc.neq(16)))

    else:

        final = mod_mtbs

    # read in the predictions and mask them with the areas
    predictions = ee.ImageCollection('projects/gee-serdp-upload/assets/cnn_mapping/mtbs_' + str(year) + '_preds_128_32').max().clip(aoi)

    #threshold at 0.5 probability
    thresholded = predictions.select('prediction').updateMask(final).updateMask(predictions.select('prediction').gte(0.5))

    #water mask
    thresholded = thresholded.updateMask(water.unmask().Not())

    #remove areas from gabam where I didn't have imagery
    gabam = gabam.updateMask(predictions)


    gabam = gabam.updateMask(lc.neq(12).And(lc.neq(16)))


    #large fire databases which is mtbs and nbac
    lfdb = ee.FeatureCollection('users/spotter/fire_cnn/raw/ak_ca_1985')

    lfdb = lfdb.filter(ee.Filter.eq("Year", year))


    #make image of fire databases
    lfdb_img = lfdb.reduceToImage(
      properties= ['Year'],
      reducer= ee.Reducer.max()).clip(aoi).select(['max'], ['constant'])
    
    #we will also want the area of ALFD as well though so do the same thing for it
    alfd = ee.FeatureCollection('users/spotter/fire_cnn/raw/ak_lfdb_1985')

    alfd = alfd.filter(ee.Filter.eq("Year", year))


    #make image of fire databases
    alfd_img = alfd.reduceToImage(
      properties= ['Year'],
      reducer= ee.Reducer.max()).clip(aoi).select(['max'], ['constant'])
    

    #water mask to our predictions
    thresholded = thresholded.updateMask(water.unmask().Not())
    
    

    #get intersection of us and mtbs
    us_int = lfdb_img.updateMask(predictions.gte(0.5)).select(['constant'], ['us_int'])

    #get omission of us and mtbs (mtbs has fire we do not)
    us_om = lfdb_img.updateMask(thresholded.unmask().Not()).select(['constant'], ['us_om'])

    #get comission of us and mtbs  (we have fire but mtbs does not)
    us_com = thresholded.updateMask(lfdb_img.unmask().Not()).select(['prediction'], ['us_com'])

    #get intersection of gabam and mtbs
    gabam_int = lfdb_img.updateMask(gabam).select(['constant'], ['gabam_int'])

    #get omission of gabam and mtbs
    gabam_om = lfdb_img.updateMask(gabam.unmask().Not()).select(['constant'], ['gabam_om'])

    #get comission of gabam and mtbs
    gabam_com = gabam.updateMask(lfdb_img.unmask().Not()).select(['b1'], ['gabam_com'])

    
    #all grid cells to loop through for area calculations
    all_territories = ee.List(grd.distinct(["Id"]).aggregate_array("Id")).getInfo()

    for this_id in all_territories:
        
        #full pathway to images within the collection
        fname = 'om_com_' + str(year) + '_' + str(this_id)
        
        #check if output exists
        bucket_name = 'smp-scratch'
        storage_client = storage.Client()

        bucket = storage_client.bucket(bucket_name)

        stats = storage.Blob(bucket=bucket, name=fname).exists(storage_client)
        
        if stats == False:
            
            #filter sub grid cell of interest
            sub_id = grd.filter(ee.Filter.eq('Id',  this_id))

            #now get areas and convert to feature collection and send out to csv
            #now get the areas
            pa = ee.Image.pixelArea()

            #areas are in m2
            
            #this is mtbs
            img_area = pa.updateMask(lfdb_img).reduceRegion(
                reducer= ee.Reducer.sum(),
                geometry= sub_id,
                scale= 30,
                tileScale= 16,
                crs= 'EPSG:3413',
                # maxPixels= 1e9,
                bestEffort=False,
                maxPixels= 1e13
            ).get('area')
            
            #alfd
            alfd_area = pa.updateMask(alfd_img).reduceRegion(
                reducer= ee.Reducer.sum(),
                geometry= sub_id,
                scale= 30,
                tileScale= 16,
                crs= 'EPSG:3413',
                # maxPixels= 1e9,
                bestEffort=False,
                maxPixels= 1e13
            ).get('area')

            us_int_area = pa.updateMask(us_int).reduceRegion(
                reducer= ee.Reducer.sum(),
                geometry= sub_id,
                scale= 30,
                tileScale= 16,
                crs= 'EPSG:3413',
                # maxPixels= 1e9,
                bestEffort=False,
                maxPixels= 1e13
            ).get('area')

            us_om_area = pa.updateMask(us_om).reduceRegion(
                reducer= ee.Reducer.sum(),
                geometry= sub_id,
                scale= 30,
                tileScale= 16,
                crs= 'EPSG:3413',
                # maxPixels= 1e9,
                bestEffort=False,
                maxPixels= 1e13
              ).get('area')

            us_com_area = pa.updateMask(us_com).reduceRegion(
                reducer= ee.Reducer.sum(),
                geometry= sub_id,
                scale= 30,
                tileScale= 16,
                crs= 'EPSG:3413',
                # maxPixels= 1e9,
                bestEffort=False,
                maxPixels= 1e13
            ).get('area')

            gabam_int_area = pa.updateMask(gabam_int).reduceRegion(
                reducer= ee.Reducer.sum(),
                geometry= sub_id,
                scale= 30,
                tileScale= 16,
                crs= 'EPSG:3413',
                # maxPixels= 1e9,
                bestEffort=False,
                maxPixels= 1e13
            ).get('area')

            gabam_om_area = pa.updateMask(gabam_om).reduceRegion(
                reducer= ee.Reducer.sum(),
                geometry= sub_id,
                scale= 30,
                tileScale= 16,
                crs = 'EPSG:3413',
                # maxPixels= 1e9,
                bestEffort=False,
                maxPixels= 1e13
              ).get('area')

            gabam_com_area = pa.updateMask(gabam_com).reduceRegion(
                reducer= ee.Reducer.sum(),
                geometry= sub_id,
                scale= 30,
                tileScale= 16,
                crs= 'EPSG:3413',
                # maxPixels= 1e9,
                bestEffort=False,
                maxPixels= 1e13
            ).get('area')

            
            #get a feature for each cire area with appropriate attributes
            us_int_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Int', 'Area': us_int_area})
            us_om_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Om', 'Area': us_om_area})
            us_com_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Us_Com', 'Area': us_com_area})
            gabam_int_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Int', 'Area': gabam_int_area})
            gabam_om_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Om', 'Area': gabam_om_area})
            gabam_com_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'Gabam_Com', 'Area': gabam_com_area})
            mtbs_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'MTBS_Area', 'Area': img_area})
            alfd_feat = ee.Feature(None, {'Year': year, 'ID': this_id, 'Class': 'ALFD_Area', 'Area': img_area})

                
            #combine to feature collection
            final_sub = ee.FeatureCollection([us_int_feat, us_om_feat, us_com_feat, gabam_int_feat, gabam_om_feat, gabam_com_feat, mtbs_feat, alfd_feat])
            
            #download
            print('Downloading ' +  'om_com_' + str(year) + '_' + str(this_id))
            
            task = ee.batch.Export.table.toCloudStorage(collection = final_sub, 
                                        description = fname,
                                        bucket =  'smp-scratch', 
                                        fileFormat = 'CSV')

            task.start()




NameError: name 'grd' is not defined